In [1]:
###################################################################

# ********* Model Inference V1.1: model.pkl input system *********
# created by tongdun.net
# This is used for sklearn inference
# ***********************introduction******************************
# 模型推理文件内含：config.json[工程内含config解析]
# ****************************Note*********************************
# 注意1 不要在代码中使用try，否则系统无法获取异常信息
# 注意2 系统调用时调用Model().predict(data)方法,不要改predict传入方法个数,自定义代码在predict修改,其他不需要修改
# 输入输出格式都为[{key1:value1,key2:value2}] 多个{}表示多组输入输出数据
# 比如输入 [{"x1":1.0, "x2": 23, "x3": 6091202002, "x4": 51, "x5": 5103, "x6": 510322, "x7": 2}]
# 输出 [{"probability": [0.5677090805330596, 0.43229091946694065], "target": 0}]
# 其中probabilityh和target都是在上传模型时设置的输出参数#####################################################################
# 具体例子可看if __name__ == "__main__":

####################################################################
import sys
import os
import warnings
import joblib
import simplejson as json
import json
import pandas as pd
import numpy as np
import time

realpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(realpath)  # 引用当前路径下模块
warnings.filterwarnings("ignore")

NameError: name '__file__' is not defined

In [ ]:
# 提取当前文件相对路径,为了兼容模型服务运行时获取模型构件
filepath = os.path.split(os.path.realpath(__file__))[0]
print("filepath：" + filepath)
inputs = json.load(open(filepath + "/config.json", "rb"))
print("used_model_file：" + inputs['files'][0])
"""outputType模型类型：分类(classifier2，classifier_more)、回归(regression)、聚类(cluster)、自定义"""
outputType = inputs['outputType']
print("outputType:" + outputType)
outputs = inputs['parameters']['outputs']
print("size of outputs:", len(inputs['parameters']['outputs']))
params_name = []
for param_temp in inputs['parameters']['inputs']:
    params_name.append(param_temp['name'])
print("param_name:", params_name)


In [ ]:
def list_json(lst):
    key_list = []
    for name in outputs:
        key_list.append(name['name'])
    info = dict(zip(key_list, lst))
    arr = np.array(pd.DataFrame(info)).tolist()
    result_info = [dict(zip(key_list, temp)) for temp in arr]
    str_json = json.dumps(result_info, ensure_ascii=False)  # json转为string
    return str_json


# 约定类名(Model),不需要修改.
class Model(object):
    def __init__(self):
        # 载入模型文件
        model = joblib.load(filepath + "/" + inputs['files'][0])
        self.xgb_model = model

    # 自定义脚本在predit中修改
    def predict(self, data=[]):
        """params输入数据包含:
            data: 模型入参 [{key:value},{}]  API:[{key:value}] 批量：[{key:value},{...}]
       """
        input_matrix = pd.DataFrame.from_dict(data, orient='columns')
        input_matrix = input_matrix[params_name]
        if outputType in ["classifier2", "classifier_more"]:
            # 拿第一条数据试运行下
            self.xgb_model.predict_proba(input_matrix)
            return list_json(
                (self.xgb_model.predict_proba(input_matrix).tolist(), self.xgb_model.predict(input_matrix).tolist()))
        elif outputType == "regression":
            return list_json(self.xgb_model.predict(input_matrix).tolist())


# 测试方法
def test_prepare(test_data=[], start_time=time.clock()):
    """ test_data格式
    [
        {key1:value1,key2:value2},
        {key1:value3,key2:value4},
        ...
    ]
    """

    # 测试数据
    if len(test_data) == 0:
        data_dict = {}
        for param in inputs['parameters']['inputs']:
            data_dict[param['name']] = param["defaultValue"]
        test_data.append(data_dict)
        test_data.append(data_dict)  # 添加两遍是为了检测批量处理的正确性

    # print("{0:>20} {1:}".format("Request:", test_data)) # 日志输出可能过长不易观察(可手动打开注释)
    result = Model().predict(test_data)  # 推理预测
    print("{0:>20} {1:}".format("Response:", result), "cost:", time.clock() - start_time, "ms")

    result = result if isinstance(result, list) else json.loads(result)  # 格式化类型: string > list
    if test_data.__len__() != result.__len__():
        print("ERROR: 模型入参({})数量与出参({})数量不一致.".format(test_data.__len__(), result.__len__()))

In [ ]:

""" README
结构规约：推理代码中必须按照当前框架.含Model定义的class,使用请详见示例.
执行过程: 载入当前文件(model_server.py) > 加载上下文 > 初始化class Model(),执行__init__ > 再执行 predict(data)
"""

import sys
import os
import warnings
import json
import joblib
import time

realpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(realpath)  # 引用当前路径下模块

### 引用文件py文件示例
# if not __package__:
#     import import_func
# else :
#     from . import import_func

warnings.filterwarnings("ignore")  # 忽略警告

# 提取当前文件相对路径,为了兼容模型服务运行时获取模型构件
filepath = os.path.split(os.path.realpath(__file__))[0]
print("{0:>20} {1:}".format("filepath:", filepath))

# 预初始Config对象
config = json.load(open(filepath + "/config.json", "rb"))
print("{0:>20} {1:}".format("config.json：", json.dumps(config, ensure_ascii=False), chr(12288)))


# ------------------------- 自定义推理代码区 ----------------------
# 约定类名(Model),不需要修改.
class Model(object):

    # 初始化: 加载模型,预处理
    def __init__(self):
        """ Describe preprocessor and model. """
        self.ml_model = joblib.load(filepath + "/" + config['files'][0])  # 示例:加载元模型

    # 模型推理代码
    def predict(self, data=[]):
        """
        :param data: 模型入参 [{key:value},{}]
        场景: 在线预测:[{key:value}] 批量预测：[{key:value},{}...]
        """
        result = []
        for item in data: result.append({"sum": item["x1"] + item["x2"]})
        return result


# -------------------------test method----------------------
def test_prepare(test_data=[], start_time=time.clock()):
    """ test_data格式
    [
        {key1:value1,key2:value2},
        {key1:value3,key2:value4},
        ...
    ]
    """

    # 测试数据
    if len(test_data) == 0:
        data_dict = {}
        for param in config['parameters']['inputs']:
            data_dict[param['name']] = param["defaultValue"]
        test_data.append(data_dict)
        test_data.append(data_dict)  # 添加两遍是为了检测批量处理的正确性

    # print("{0:>20} {1:}".format("Request:", test_data)) # 日志输出可能过长不易观察(可手动打开注释)
    result = Model().predict(test_data)  # 推理预测
    print("{0:>20} {1:}".format("Response:", result), "cost:", time.clock() - start_time, "ms")

    result = result if isinstance(result, list) else json.loads(result)  # 格式化类型: string > list
    if test_data.__len__() != result.__len__():
        print("ERROR: 模型入参({})数量与出参({})数量不一致.".format(test_data.__len__(), result.__len__()))


if __name__ == "__main__":
    test_prepare()

In [ ]:


###################################################################

# ********* Model Inference V1.1: model.pkl input system *********
# created by tongdun.net
# This is used for sklearn inference
# ***********************introduction******************************
# 模型推理文件内含：config.json[工程内含config解析]
# ****************************Note*********************************
# Note1：不要在代码中使用try，否则系统无法获取异常信息
# Note2：推理代码中必须按照当前框架，含Model定义的class，__init__用于定义变量
# predict输出模型结果使用：predict()或者predict_proba()，list_json用于转
# json(将输出结果转json)。 系统调用Model().predict(data) 输出结果
# 输入输出格式都为[{key1:value1,key2:value2}] 多个{}表示多组输入输出数据
# 比如输入 [{"x1":1.0, "x2": 23, "x3": 6091202002, "x4": 51, "x5": 5103, "x6": 510322, "x7": 2}]
# 输出 [{"probability": [0.5677090805330596, 0.43229091946694065], "target": 0}]
# 其中probabilityh和target都是在上传模型时设置的输出参数
# 具体例子可看if __name__ == "__main__":

####################################################################
import sys
import os
import warnings
import logging
import joblib
import simplejson as json
import json
import pandas as pd
import numpy as np
import time

realpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(realpath)  # 引用当前路径下模块

warnings.filterwarnings("ignore")

# 提取当前文件相对路径,为了兼容模型服务运行时获取模型构件
filepath = os.path.split(os.path.realpath(__file__))[0]
print("filepath：" + filepath)
inputs = json.load(open(filepath + "/config.json", "rb"))
print("used_model_file：" + inputs['files'][0])
"""outputType模型类型：分类(classifier2，classifier_more)、回归(regression)、聚类(cluster)、自定义"""
outputType = inputs['outputType']
print("outputType:" + outputType)
outputs = inputs['parameters']['outputs']
print("size of outputs:", len(inputs['parameters']['outputs']))
params_name = []
for param_temp in inputs['parameters']['inputs']:
    params_name.append(param_temp['name'])
print("param_name:", params_name)


def list_json(lst):
    key_list = []
    for name in outputs:
        key_list.append(name['name'])
    info = dict(zip(key_list, lst))
    arr = np.array(pd.DataFrame(info)).tolist()
    result_info = [dict(zip(key_list, temp)) for temp in arr]
    str_json = json.dumps(result_info, ensure_ascii=False)  # json转为string
    return str_json
# 约定类名(Model),不需要修改.
class Model(object):
    def __init__(self):
        # 载入模型文件
        model = joblib.load(filepath + "/" + inputs['files'][0])
        self.ml_model = model

    def predict(self, data):
        """params输入数据包含:
            data: 模型入参 [{key:value},{}]  API:[{key:value}] 批量：[{key:value},{...}]、
        """
        # 列表转化为DataFrame
        param = pd.DataFrame.from_dict(data, orient='columns')
        param = param[params_name]
        if outputType in ["classifier2", "classifier_more"]:
            try:
                # 拿第一条数据试运行下
                self.ml_model.predict_proba(param)
            except AttributeError:
                logging.error("请选择正确的分类模型")
            else:
                return list_json((self.ml_model.predict_proba(param).tolist(), self.ml_model.predict(param).tolist()))
        elif outputType == "regression":
            return list_json([self.ml_model.predict(param).tolist()])
        elif outputType == "cluster":
            return list_json([self.ml_model.predict(param).tolist()])
        else:
            logging.info("自定义模型类型")


# -------------------------test method----------------------
def test_prepare(test_data=[], start_time=time.clock()):
    """ test_data格式
    [
        {key1:value1,key2:value2},
        {key1:value3,key2:value4},
        ...
    ]
    """

    # 测试数据
    if len(test_data) == 0:
        data_dict = {}
        for param in inputs['parameters']['inputs']:
            data_dict[param['name']] = param["defaultValue"]
        test_data.append(data_dict)
        test_data.append(data_dict)  # 添加两遍是为了检测批量处理的正确性

    # print("{0:>20} {1:}".format("Request:", test_data)) # 日志输出可能过长不易观察(可手动打开注释)
    result = Model().predict(test_data)  # 推理预测
    print("{0:>20} {1:}".format("Response:", result), "cost:", time.clock() - start_time, "ms")

    result = result if isinstance(result, list) else json.loads(result)  # 格式化类型: string > list
    if test_data.__len__() != result.__len__():
        print("ERROR: 模型入参({})数量与出参({})数量不一致.".format(test_data.__len__(), result.__len__()))


if __name__ == "__main__":
    test_prepare()
